In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os
import time

In [ ]:
path = '/home/gordo/development/Projects/web_scraping/Selenium_Projects/chromedriver'
web = 'https://twitter.com/'

In [ ]:
driver = webdriver.Chrome(path)
driver.get(web)
driver.maximize_window()

# Login Buttom

In [ ]:
login = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//a[@href="/login"]')))
login.click()

# User Name Complete

In [ ]:
username = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//input[@autocomplete="username"]')))
username.send_keys('@allrealstate')

# Next

In [ ]:
next_button = driver.find_element_by_xpath('//div[@role="button"]//span[text()="Next"]')
next_button.click()

# Password

In [ ]:
# Xpaths Old = //input[@autocomplete="on"]
password = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//input[@autocomplete="current-password"]')))
password.send_keys('jc20385796')

# LOG IN

In [ ]:
# Old Xpaths = '//div[@role="button"]//span[text()="Next"]'
login = driver.find_element_by_xpath('.//div[@role="button"]//span[text()="Log in"]')
login.click()

# SEARCH

In [ ]:
search = WebDriverWait(driver, 55).until(EC.presence_of_element_located((By.XPATH, '//input[@aria-autocomplete="list"]')))
search.send_keys('criptomonedas')
time.sleep(10)

In [ ]:
result = WebDriverWait(driver, 55).until(EC.presence_of_element_located((By.XPATH, '//div[@data-testid="typeaheadResult"]')))
result.click()

# INFINIT SCROLLING

In [ ]:
def get_tweet(element):
    try:
        user = element.find_element_by_xpath('.//span[contains(text(),"@")]').text 
        text = element.find_element_by_xpath('.//div[@lang]').text
        tweets_data = [user, text]
        return tweets_data
    except:
        tweets_data = ['user', 'data']
        return tweets_data

In [ ]:
user_data = []
text_data = []
tweets_id = set()
scrolling = True

while scrolling:
    tweets = WebDriverWait(driver, 25).until(EC.presence_of_all_elements_located((By.XPATH, '//article[@role="article"]')))
    for tweet in tweets[-15:]:
        tweet_list = get_tweet(tweet)
        tweet_id = "".join(tweet_list)
        if tweet_id not in tweets_id:
            tweets_id.add(tweet_id)
        user_data.append(tweet_list[0])
        text_data.append(" ".join(tweet_list[1].split()))

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        # ALL PAGES SCRAPE
        if new_height == last_height:
            scrolling = False
            break
        # DATA LIMIT (TWEETS) TO SCRAPE
        # if len(user_data) > 100:
            # scrolling = False
            # break
        else:
            last_height = new_height
            break



In [ ]:
df_tweets = pd.DataFrame({'User':user_data, 'Tweet':text_data})
df_tweets.to_csv('tweets_2.csv', index=False, encoding= 'utf8')

In [ ]:
driver.quit()